<h1>Bonus 2  M11007309 鄭維新<h1>

<h3>在程式中我先用了Python的爬蟲去抓取FB社團內成員的名子
然後再對教授提供的資料去進行操作<h3>
<h3>因為要爬資訊，所以執行上可能會有一點久(不超過1分鐘)，程式執行到最後會輸出一個EXCEL，裡面就包含了教授要求的三個問題<h3>
<h3>Note : 這次程式如果FB是英文名稱的話會無法去辨識 <h3>

In [42]:
# 爬取fb社團成員
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import os
import wget
import pandas as pd

In [43]:
#open facebook
PATH = "chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.facebook.com/groups/189644259942426/members")

"""
使用chromedriver需要注意版號，需要測試請查看自己的chrome版本後到 https://chromedriver.chromium.org/downloads 找到對應自己版號的chromedriver進行下載
路徑也要自行設定，可以跟程式放在同一個資料夾
"""

'\n使用chromedriver需要注意版號，需要測試請查看自己的chrome版本後到 https://chromedriver.chromium.org/downloads 找到對應自己版號的chromedriver進行下載\n路徑也要自行設定，可以跟程式放在同一個資料夾\n'

In [44]:
#login to facebook
username = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "email"))
)
password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "pass"))
)
login = driver.find_element_by_xpath('//*[@id="loginbutton"]')

username.clear()
password.clear()
username.send_keys('FB帳號')
password.send_keys('FB密碼')
login.click()

time.sleep(20)
homepage = driver.find_element(By.ID, 'facebook')
webdriver.ActionChains(driver).click(homepage).perform()

In [45]:
#Get fb member numbers
member_title = driver.find_element_by_class_name('rq0escxv.l9j0dhe7.du4w35lb.j83agx80.pfnyh3mw.jifvfom9.bp9cbjyn.lhclo0ds.btwxx1t3.jb3vyjys.nkwizq5d.roh60bw9.scwd0bx6.hop8lmos')
member_title_elements = member_title.find_elements(By.TAG_NAME, 'div')
member_text = str(member_title_elements[5].text)
member_number = int(member_text.replace(" members",""))

54


In [46]:
#scroll down to load more page
for i in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

In [47]:
#get all members in group
fb_member_list = []
for i in range(1,member_number+1):
    member_xpath = "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[4]/div/div/div/div/div/div/div/div/div/div/div[2]/div[12]/div/div[2]/div/div["+str(i)+"]/div/div/div[2]/div[1]/div/div/div[1]/span/span/span/a"
    fb_member_list.append(driver.find_element_by_xpath(member_xpath).text)
fb_member_list.sort()
fb_member_list.remove('Sheng-Luen Chung') #移除教授的名子，因為教授不是學生

In [48]:
file = r'1101_EE5327701_4242_1006.xls'
df = pd.read_excel(file)
register_list = list(df['姓名'])
register_list.sort()

In [49]:
register_list?

Type:        list
String form: ['侯宏林', '劉冠甫', '劉錕笙', '吳芊瑩', '廖英廷', '張仕嘉', '張凱鈞', '張嘉瑩', '張詠勝', '曾家威', '朱揚傑', '李榮昌', '李汭璇', '李秉和' <...> '邱博宏', '邱心漁', '郭仲瑾', '郭威霆', '鄭維新', '陳冠宇', '陳威澤', '陳學勤', '陳文祥', '陳苡昕', '陳靜萱', '韓宗翰', '黃旭輝', '黃翔莉']
Length:      42
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


In [50]:
fb_member_list?

Type:        list
String form: ['Bread Chen', 'CF Chien', 'Eric Tsai', 'Jinde Lin', 'Joseph Wu', 'Karen Chow', 'Kuan-yu Chen', ' <...> '邱心漁', '郭仲瑾', '郭威霆', '鄭維新', '陳威澤', '陳廷嘉', '陳文祥', '陳柏華', '陳苡昕', '陳靜萱', '韓宗翰', '高郁芩', '黃旭輝', '黃翔莉']
Length:      53
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


In [51]:
all_member = [] #將fb社團成員和註冊名單合併
fb_not_in_register_list = [] #fb社團成員不在註冊名單內
register_not_in_fb_list = [] #註冊名單不在fb社團內

#將fb社團成員和註冊名單合併
all_member = fb_member_list.copy()
for i in range(len(register_list)):
    all_member.insert(0,register_list[i])
all_member.sort()
set(all_member)

#fb社團成員不在註冊名單內
for i in range(len(fb_member_list)):
    if(not fb_member_list[i] in register_list):
        fb_not_in_register_list.append(fb_member_list[i])

#註冊名單不在fb社團內
for i in range(len(register_list)):
    if(not register_list[i] in fb_member_list):
        register_not_in_fb_list.append(register_list[i])

In [52]:
#Mapping
mapping_member = [[0 for x in range(len(all_member))] for y in range(2)]

j = len(fb_member_list)
for i in range(len(fb_member_list)):
    mapping_member[0][i] = fb_member_list[i]
for i in range(len(register_list)):
    if(register_list[i] in fb_member_list):
        mapping_member[1][fb_member_list.index(register_list[i])] = register_list[i]
    else:
        mapping_member[1][j] = register_list[i]
        j+=1

In [53]:
import xlsxwriter

workbook = xlsxwriter.Workbook('output.xlsx')
worksheet = workbook.add_worksheet()

# Widen the column to make the text clearer
worksheet.set_column('A:B', 20)
worksheet.set_column('C:D', 30)

# Write column name
worksheet.write('A1', 'in FB')
worksheet.write('B1', 'in registered list')
worksheet.write('C1', 'Registered list not in FB group')
worksheet.write('D1', 'FB member not in registered list')

for i in range(len(mapping_member)):
    for j in range(len(all_member)):
        if (mapping_member[i][j] == 0):
            worksheet.write(j+1, i, "")
        else:
            worksheet.write(j+1, i, mapping_member[i][j])

for i in range(len(register_not_in_fb_list)):
    worksheet.write(i+1, 2, register_not_in_fb_list[i])

for i in range(len(fb_not_in_register_list)):
    worksheet.write(i+1, 3, fb_not_in_register_list[i])

workbook.close()

In [54]:
"""ANSWER"""

#Mapping between fb and registered list
print("Mapping between fb and registered list")
print(mapping_member)


#Registered students but have not joined this fb group
print("Registered students but have not joined this fb group")
print(register_not_in_fb_list)


#FB member but not registered in the class
print("FB member but not registered in the class")
print(fb_not_in_register_list)


Mapping between fb and registered list
[['Bread Chen', 'CF Chien', 'Eric Tsai', 'Jinde Lin', 'Joseph Wu', 'Karen Chow', 'Kuan-yu Chen', 'Ren Jhang', 'Scott Chang', 'YK Tsai', '劉錕笙', '吳芊瑩', '張凱鈞', '張詠勝', '朱揚傑', '李榮昌', '李汭璇', '李秉和', '李維', '李采玨', '林妮蓁', '林芷因', '楊可東', '楊喬鈞', '楊竣崴', '沈修民', '沈育民', '洪雅玟', '潘乃聿', '王小禾', '王琪惠', '王璽鈞', '石家安', '許揚', '謝昌哲', '賴俊維', '賴嘉晟', '賴昱瑋', '邱博宏', '邱心漁', '郭仲瑾', '郭威霆', '鄭維新', '陳威澤', '陳廷嘉', '陳文祥', '陳柏華', '陳苡昕', '陳靜萱', '韓宗翰', '高郁芩', '黃旭輝', '黃翔莉', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '劉錕笙', '吳芊瑩', '張凱鈞', '張詠勝', '朱揚傑', '李榮昌', '李汭璇', '李秉和', '李維', 0, 0, '林芷因', 0, 0, '楊竣崴', 0, '沈育民', '洪雅玟', '潘乃聿', '王小禾', '王琪惠', '王璽鈞', 0, 0, 0, '賴俊維', 0, 0, '邱博宏', '邱心漁', '郭仲瑾', '郭威霆', '鄭維新', '陳威澤', 0, '陳文祥', 0, '陳苡昕', '陳靜萱', '韓宗翰', 0, '黃旭輝', '黃翔莉', '侯宏林', '劉冠甫', '廖英廷', '張仕嘉', '張嘉瑩', '曾家威', '李采珏', '林晉德', '蔡卓倫', '蔡沅格', '陳冠宇', '陳學勤', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 